In [1]:
from sklearn.model_selection import train_test_split
import os
from PIL import Image
import numpy as np
import random
import tensorflow as tf
import keras
from keras.utils import to_categorical

In [2]:
batch_size = 32
img_height = 224
img_width = 224

In [3]:
import cv2
from sklearn.model_selection import train_test_split

In [4]:
(X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()

# Preprocess the data
X_train = X_train.reshape(X_train.shape[0], 28, 28)
X_test = X_test.reshape(X_test.shape[0], 28, 28)
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255
x_train = np.stack([X_train, X_train, X_train], axis=-1)
X_test = np.stack([X_test, X_test, X_test], axis=-1)

# Convert class vectors to binary class matrices
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)


11490434/11490434 [==============================] - 0s 0us/step


In [5]:
print("Shape of X_train:", X_train.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_test:", y_test.shape)

Shape of X_train: (60000, 28, 28)
Shape of y_train: (60000, 10)
Shape of X_test: (10000, 28, 28, 3)
Shape of y_test: (10000, 10)


In [6]:
import numpy as np

import numpy as np

def top_k_accuracy(predictions, labels, k=1):
    """
    Calculate the top-k accuracy.

    Args:
    predictions: Array-like, predicted binary values (0 or 1).
    labels: Array-like, ground truth binary values (0 or 1).
    k: Integer, denotes top-k accuracy.

    Returns:
    accuracy: Float, top-k accuracy.
    """
    sorted_indices = np.argsort(predictions, axis=1)[:, ::-1]
    top_k_indices = sorted_indices[:, :k]
    top_k_labels = np.take_along_axis(labels, top_k_indices, axis=1)
    correct_predictions = np.sum(top_k_labels, axis=1)
    accuracy = np.mean(correct_predictions)
    return accuracy


def top_1_accuracy(predictions, labels):
    """
    Calculate the top-1 accuracy.

    Args:
    predictions: Array-like, predicted binary values (0 or 1).
    labels: Array-like, ground truth binary values (0 or 1).

    Returns:
    accuracy: Float, top-1 accuracy.
    """
    return top_k_accuracy(predictions, labels, k=1)

def top_5_accuracy(predictions, labels):
    """
    Calculate the top-5 accuracy.

    Args:
    predictions: Array-like, predicted binary values (0 or 1).
    labels: Array-like, ground truth binary values (0 or 1).

    Returns:
    accuracy: Float, top-5 accuracy.
    """
    return top_k_accuracy(predictions, labels, k=5)

def top_k_error(predictions, labels, k=1):
    """
    Calculate the top-k error rate.

    Args:
    predictions: Array-like, predicted binary values (0 or 1).
    labels: Array-like, ground truth binary values (0 or 1).
    k: Integer, denotes top-k error.

    Returns:
    error: Float, top-k error rate.
    """
    return 1.0 - top_k_accuracy(predictions, labels, k)

def top_1_error(predictions, labels):
    """
    Calculate the top-1 error rate.

    Args:
    predictions: Array-like, predicted binary values (0 or 1).
    labels: Array-like, ground truth binary values (0 or 1).

    Returns:
    error: Float, top-1 error rate.
    """
    return top_k_error(predictions, labels, k=1)

def top_5_error(predictions, labels):
    """
    Calculate the top-5 error rate.

    Args:
    predictions: Array-like, predicted binary values (0 or 1).
    labels: Array-like, ground truth binary values (0 or 1).

    Returns:
    error: Float, top-5 error rate.
    """
    return top_k_error(predictions, labels, k=5)

In [7]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from keras.models import load_model


In [9]:

stage2 = load_model("/content/drive/MyDrive/Trained Models/MNIST/resnet18_with_cbam_after_stage_2_MNIST_val_accuracy.h5")
y_pred = stage2.predict(X_test)

threshold = 0.5
y_pred_binary = np.where(y_pred > threshold, 1, 0)
accuracy = accuracy_score(y_test, y_pred_binary)
print("Accuracy:", accuracy)
print("Top-1 Accuracy:", top_1_accuracy(y_pred_binary, y_test))
print("Top-5 Accuracy:", top_5_accuracy(y_pred_binary, y_test))
print("Top-1 Error:", top_1_error(y_pred_binary, y_test))
print("Top-5 Error:", top_5_error(y_pred_binary, y_test))
print("Classification Report:")
print(classification_report(y_test, y_pred_binary))

313/313 [==============================] - 25s 78ms/step
Accuracy: 0.9945
Top-1 Accuracy: 0.9945
Top-5 Accuracy: 0.9975
Top-1 Error: 0.00550001859664917
Top-5 Error: 0.002499997615814209
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       980
           1       1.00      1.00      1.00      1135
           2       1.00      1.00      1.00      1032
           3       0.99      0.99      0.99      1010
           4       0.99      0.99      0.99       982
           5       0.99      0.99      0.99       892
           6       1.00      0.99      1.00       958
           7       0.99      0.99      0.99      1028
           8       0.99      0.99      0.99       974
           9       0.99      0.99      0.99      1009

   micro avg       0.99      0.99      0.99     10000
   macro avg       0.99      0.99      0.99     10000
weighted avg       0.99      0.99      0.99     10000
 samples avg       0.99      0.9

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [10]:
stage3 = load_model("/content/drive/MyDrive/Trained Models/MNIST/resnet18_with_cbam_after_stage_3_MNIST_val_accuracy.h5")
y_pred = stage3.predict(X_test)
threshold = 0.5
y_pred_binary = np.where(y_pred > threshold, 1, 0)
accuracy = accuracy_score(y_test, y_pred_binary)
print("Accuracy:", accuracy)
print("Top-1 Accuracy:", top_1_accuracy(y_pred_binary, y_test))
print("Top-5 Accuracy:", top_5_accuracy(y_pred_binary, y_test))
print("Top-1 Error:", top_1_error(y_pred_binary, y_test))
print("Top-5 Error:", top_5_error(y_pred_binary, y_test))
print("Classification Report:")
print(classification_report(y_test, y_pred_binary))

313/313 [==============================] - 25s 78ms/step
Accuracy: 0.9953
Top-1 Accuracy: 0.9953
Top-5 Accuracy: 0.9979
Top-1 Error: 0.004700005054473877
Top-5 Error: 0.0020999908447265625
Classification Report:
              precision    recall  f1-score   support

           0       0.99      1.00      1.00       980
           1       1.00      1.00      1.00      1135
           2       1.00      1.00      1.00      1032
           3       0.99      1.00      0.99      1010
           4       1.00      1.00      1.00       982
           5       1.00      0.99      0.99       892
           6       1.00      0.99      0.99       958
           7       0.99      0.99      0.99      1028
           8       1.00      1.00      1.00       974
           9       0.99      0.99      0.99      1009

   micro avg       1.00      1.00      1.00     10000
   macro avg       1.00      1.00      1.00     10000
weighted avg       1.00      1.00      1.00     10000
 samples avg       1.00      1

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [11]:
stage4 = load_model("/content/drive/MyDrive/Trained Models/MNIST/resnet18_with_cbam_after_stage_4_MNIST_val_accuracy.h5")
y_pred = stage4.predict(X_test)
threshold = 0.5
y_pred_binary = np.where(y_pred > threshold, 1, 0)
accuracy = accuracy_score(y_test, y_pred_binary)
print("Accuracy:", accuracy)
print("Top-1 Accuracy:", top_1_accuracy(y_pred_binary, y_test))
print("Top-5 Accuracy:", top_5_accuracy(y_pred_binary, y_test))
print("Top-1 Error:", top_1_error(y_pred_binary, y_test))
print("Top-5 Error:", top_5_error(y_pred_binary, y_test))
print("Classification Report:")
print(classification_report(y_test, y_pred_binary))

313/313 [==============================] - 26s 82ms/step
Accuracy: 0.9944
Top-1 Accuracy: 0.9944
Top-5 Accuracy: 0.9976
Top-1 Error: 0.0055999755859375
Top-5 Error: 0.0023999810218811035
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       980
           1       0.99      1.00      1.00      1135
           2       1.00      0.99      1.00      1032
           3       0.99      1.00      0.99      1010
           4       0.99      1.00      0.99       982
           5       1.00      0.99      0.99       892
           6       1.00      0.99      0.99       958
           7       0.99      1.00      0.99      1028
           8       0.99      1.00      1.00       974
           9       1.00      0.99      0.99      1009

   micro avg       0.99      0.99      0.99     10000
   macro avg       0.99      0.99      0.99     10000
weighted avg       0.99      0.99      0.99     10000
 samples avg       0.99      0.9

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [12]:
stage5 = load_model("/content/drive/MyDrive/Trained Models/MNIST/resnet18_with_cbam_after_stage_5_MNIST_val_accuracy.h5")
y_pred = stage5.predict(X_test)
threshold = 0.5
y_pred_binary = np.where(y_pred > threshold, 1, 0)
accuracy = accuracy_score(y_test, y_pred_binary)
print("Accuracy:", accuracy)
print("Top-1 Accuracy:", top_1_accuracy(y_pred_binary, y_test))
print("Top-5 Accuracy:", top_5_accuracy(y_pred_binary, y_test))
print("Top-1 Error:", top_1_error(y_pred_binary, y_test))
print("Top-5 Error:", top_5_error(y_pred_binary, y_test))
print("Classification Report:")
print(classification_report(y_test, y_pred_binary))

313/313 [==============================] - 26s 80ms/step
Accuracy: 0.9939
Top-1 Accuracy: 0.9939
Top-5 Accuracy: 0.9972
Top-1 Error: 0.006099998950958252
Top-5 Error: 0.00279998779296875
Classification Report:
              precision    recall  f1-score   support

           0       0.99      1.00      1.00       980
           1       1.00      1.00      1.00      1135
           2       1.00      0.99      0.99      1032
           3       1.00      1.00      1.00      1010
           4       0.99      0.99      0.99       982
           5       1.00      0.99      0.99       892
           6       1.00      0.99      0.99       958
           7       0.99      1.00      0.99      1028
           8       0.99      1.00      1.00       974
           9       1.00      0.99      0.99      1009

   micro avg       0.99      0.99      0.99     10000
   macro avg       0.99      0.99      0.99     10000
weighted avg       0.99      0.99      0.99     10000
 samples avg       0.99      0.9

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [8]:
stageEvery = load_model("/content/drive/MyDrive/Trained Models/MNIST/resnet18_with_cbam_after_every_stage_MNIST_val_accuracy.h5")
y_pred = stageEvery.predict(X_test)
threshold = 0.5
y_pred_binary = np.where(y_pred > threshold, 1, 0)
accuracy = accuracy_score(y_test, y_pred_binary)
print("Accuracy:", accuracy)
print("Top-1 Accuracy:", top_1_accuracy(y_pred_binary, y_test))
print("Top-5 Accuracy:", top_5_accuracy(y_pred_binary, y_test))
print("Top-1 Error:", top_1_error(y_pred_binary, y_test))
print("Top-5 Error:", top_5_error(y_pred_binary, y_test))
print("Classification Report:")
print(classification_report(y_test, y_pred_binary))

313/313 [==============================] - 35s 94ms/step
Accuracy: 0.9948
Top-1 Accuracy: 0.9949
Top-5 Accuracy: 0.9977
Top-1 Error: 0.0051000118255615234
Top-5 Error: 0.0023000240325927734
Classification Report:
              precision    recall  f1-score   support

           0       0.99      1.00      1.00       980
           1       1.00      1.00      1.00      1135
           2       0.99      1.00      0.99      1032
           3       0.99      0.99      0.99      1010
           4       0.99      1.00      1.00       982
           5       0.99      0.99      0.99       892
           6       1.00      0.99      0.99       958
           7       1.00      1.00      1.00      1028
           8       1.00      0.99      0.99       974
           9       0.99      0.99      0.99      1009

   micro avg       0.99      0.99      0.99     10000
   macro avg       0.99      0.99      0.99     10000
weighted avg       1.00      0.99      0.99     10000
 samples avg       0.99      

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
